In [1]:
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from torch.autograd import Variable
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
from torch.nn import DataParallel

import time
import os
import numpy as np
import json
import cv2
from PIL import Image, ImageOps
import random
from tqdm import tqdm
import operator
import itertools
from scipy.io import  loadmat
import logging
from scipy import signal

from utils import data_transforms
from utils import get_paste_kernel, kernel_map
from utils_logging import setup_logger

from models.gazenet import GazeNet
from dataloading import GazeDataset
from models.__init__ import save_checkpoint, resume_checkpoint
from training.train_gazenet import train, test
from training.train_gazenet import StagedOptimizer

In [2]:
logger = setup_logger(name='first_logger', 
                      log_dir ='./logs/',
                      log_file='train.log',
                      log_format = '%(asctime)s %(levelname)s %(message)s',
                      verbose=True)

In [3]:
#logger.info("Log test if working again")

In [4]:
#main
    
batch_size = 32

train_set = GazeDataset(root_dir='/home/eee198/Documents/datasets/GazeFollowData/',
                        mat_file='/home/eee198/Documents/datasets/GazeFollowData/train_annotations.mat',
                        training='train')
train_data_loader = DataLoader(train_set, batch_size=batch_size,
                               shuffle=True, num_workers=16)

test_set = GazeDataset(root_dir='/home/eee198/Documents/datasets/GazeFollowData/',
                       mat_file='/home/eee198/Documents/datasets/GazeFollowData/test_annotations.mat',
                       training='test')
test_data_loader = DataLoader(test_set, batch_size=batch_size//2,
                              shuffle=False, num_workers=8)

net = GazeNet()
net = DataParallel(net)
net.cuda()

DataParallel(
  (module): GazeNet(
    (face_net): ResNet(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (relu): ReLU(inplace)

In [5]:
start_epoch = 0
method = 'Adam'
learning_rate = 0.0001
max_epoch = 25

staged_opt = StagedOptimizer(net, learning_rate)
optimizer = staged_opt.get_optimizer()

resume_training = True
resume_path = './saved_models/temp/model_epoch25.pth.tar'
if resume_training :
    net, optimizer = resume_checkpoint(net, optimizer, resume_path)
    test(net, test_data_loader,logger)
    start_epoch = 25

  0%|          | 0/299 [00:00<?, ?it/s]

=> loading checkpoint './saved_models/temp/model_epoch25.pth.tar'
=>Optimizer has different parameter groups. Usually this will occur for staged optimizers (GazeNet, GazeMask)
=> loaded checkpoint './saved_models/temp/model_epoch25.pth.tar' (epoch 26)


/home/eee198/Documents/gaze-on-objects/gazefollowing/training/train_gazenet.py:110: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  map(lambda x: Variable(x.cuda(), volatile=True), [image, face_image, gaze_field, eye_position, gt_position, gt_heatmap])
loss: 0.06168, 0.11927, 0.18096
  2%|▏         | 5/299 [00:02<02:13,  2.20it/s]loss: 0.05022, 0.08001, 0.13022
loss: 0.06150, 0.17127, 0.23277
  5%|▍         | 14/299 [00:03<00:40,  7.02it/s]loss: 0.05223, 0.05644, 0.10867
loss: 0.06078, 0.09972, 0.16049
  6%|▌         | 17/299 [00:03<00:37,  7.55it/s]loss: 0.06157, 0.04784, 0.10941
loss: 0.05582, 0.04253, 0.09835
 12%|█▏        | 37/299 [00:05<00:39,  6.64it/s]loss: 0.05443, 0.04663, 0.10107
loss: 0.05910, 0.09928, 0.15837
 13%|█▎        | 40/299 [00:06<00:32,  8.01it/s]loss: 0.06144, 0.13863, 0.20007
loss: 0.05962, 0.04646, 0.10608
 14%|█▍        | 43/299 [00:06<00:29,  8.60it/s]loss: 0.05313, 0.06760, 0.12072
loss: 0.06314, 0.21691, 0.28

loss: 0.05783, 0.05266, 0.11049
 37%|███▋      | 112/299 [00:14<00:18, 10.25it/s]loss: 0.06575, 0.09278, 0.15853
loss: 0.05508, 0.10581, 0.16089
 38%|███▊      | 114/299 [00:14<00:19,  9.73it/s]loss: 0.06422, 0.15661, 0.22083
loss: 0.05597, 0.06377, 0.11974
 40%|███▉      | 119/299 [00:15<00:21,  8.54it/s]loss: 0.06403, 0.07940, 0.14343
loss: 0.04949, 0.09425, 0.14375
 43%|████▎     | 128/299 [00:16<00:19,  8.56it/s]loss: 0.06249, 0.27815, 0.34065
loss: 0.07080, 0.11541, 0.18622
 43%|████▎     | 130/299 [00:16<00:18,  9.04it/s]loss: 0.05497, 0.11762, 0.17259
loss: 0.05735, 0.08614, 0.14349
 45%|████▍     | 134/299 [00:16<00:18,  9.04it/s]loss: 0.06338, 0.07614, 0.13952
loss: 0.06775, 0.16105, 0.22879
 45%|████▌     | 136/299 [00:17<00:17,  9.08it/s]loss: 0.06244, 0.07013, 0.13257
loss: 0.05612, 0.06919, 0.12530
 46%|████▋     | 139/299 [00:17<00:17,  9.23it/s]loss: 0.06416, 0.18245, 0.24661
loss: 0.06212, 0.05670, 0.11882
 47%|████▋     | 141/299 [00:17<00:17,  9.25it/s]loss: 0.05895, 

 76%|███████▋  | 228/299 [00:27<00:08,  8.20it/s]loss: 0.06244, 0.12918, 0.19162
loss: 0.05495, 0.03696, 0.09191
 79%|███████▉  | 236/299 [00:28<00:07,  8.73it/s]loss: 0.05126, 0.01507, 0.06633
loss: 0.06385, 0.04377, 0.10762
 80%|███████▉  | 238/299 [00:28<00:06,  8.75it/s]loss: 0.06061, 0.15759, 0.21820
loss: 0.05772, 0.11474, 0.17246
 81%|████████  | 241/299 [00:29<00:06,  9.10it/s]loss: 0.06351, 0.09309, 0.15660
loss: 0.05209, 0.12179, 0.17388
 82%|████████▏ | 246/299 [00:29<00:06,  8.49it/s]loss: 0.06111, 0.11717, 0.17828
loss: 0.05941, 0.02610, 0.08551
 83%|████████▎ | 248/299 [00:29<00:05,  9.29it/s]loss: 0.06122, 0.14635, 0.20758
loss: 0.05926, 0.09346, 0.15271
 84%|████████▍ | 252/299 [00:30<00:06,  7.43it/s]loss: 0.06810, 0.21540, 0.28350
loss: 0.05063, 0.07188, 0.12251
 85%|████████▍ | 254/299 [00:30<00:05,  8.11it/s]loss: 0.06139, 0.05490, 0.11628
loss: 0.05429, 0.06503, 0.11932
 89%|████████▊ | 265/299 [00:31<00:04,  8.34it/s]loss: 0.04966, 0.05268, 0.10233
loss: 0.05980, 

In [8]:
staged_opt = StagedOptimizer(net, learning_rate)
optimizer = staged_opt.update(24)
save_path = './saved_models/temp/'
save_checkpoint(net, optimizer, 24+1, save_path)

In [6]:
staged_opt = StagedOptimizer(net, learning_rate)

for epoch in range(start_epoch, max_epoch):
    
    # Update optimizer
    optimizer = staged_opt.update(epoch)

    # Train model
    train(net, train_data_loader, optimizer, epoch, logger)

    # Save model and optimizer
    if epoch > max_epoch-5:
        save_path = './saved_models/temp/'
        save_checkpoint(net, optimizer, epoch+1, save_path)
    
    # Evaluate model
    test(net, test_data_loader, logger)

  1%|          | 32/3924 [00:14<29:58,  2.16it/s] 


KeyboardInterrupt: 